In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('loan.csv')

In [3]:
df.shape

(9578, 14)

In [4]:
df.columns

Index(['credit.policy', 'purpose', 'int.rate', 'installment', 'log.annual.inc',
       'dti', 'fico', 'days.with.cr.line', 'revol.bal', 'revol.util',
       'inq.last.6mths', 'delinq.2yrs', 'pub.rec', 'not.fully.paid'],
      dtype='object')

In [5]:
df.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,2,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,1,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,2,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,2,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,1,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from  sklearn.metrics  import  accuracy_score , precision_score , recall_score,confusion_matrix,classification_report
rf_clf = RandomForestClassifier(n_estimators=600)

In [7]:
import mlflow

mlflow.autolog()

X = df.drop('not.fully.paid',axis=1)
y = df['not.fully.paid']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 24)

adaboost_clf = AdaBoostClassifier(learning_rate = 0.5)
adaboost_clf.fit(X_train, y_train)

y_pred_train = adaboost_clf.predict(X_train)
y_pred_test = adaboost_clf.predict(X_test)

train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

2023/05/08 00:25:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2023/05/08 00:25:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/08 00:25:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2023/05/08 00:25:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '23f4ba079e6d4b269820dfc3dc8316ff', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2023/05/08 00:25:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\naror\AppData\Roaming\Python\Python38\site-packages\mlflow\models\signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
mlflow.sklearn.autolog(max_tuning_runs=None)

with mlflow.start_run():
    tuned_parameters = [{'n_estimators':[i for i in range(1, 51)]}]

    clf = GridSearchCV(
        estimator=AdaBoostClassifier(), 
        param_grid=tuned_parameters, 
        scoring='accuracy',
        cv=5,
        return_train_score=True,
        verbose=1
    )
    clf.fit(X_train, y_train)
    
    # Disabling autologging
    mlflow.sklearn.autolog(disable=True)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


2023/05/08 00:26:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\naror\AppData\Roaming\Python\Python38\site-packages\mlflow\models\signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
